# Optical Character Recognition for MNIST handwritten characters

OCR (Optical Character Recognition) is another classification problem. In this example, we wish to recognise hand-written digits from the famous NIST dataset, each of which is presented as an $8\times 8$ array of pixel intensities.

We are just going to focus on the problem of classifying each rasterised digit scan, and not on the other steps which include tokenising text, basic data cleaning etc.

scikit-learn includes a built-in set of pre-formatted digits which we can use. The set is actually the MNIST (Modified NIST) set, but is functionally equivalent to the original NIST set, in relation to its classification challenges.

## Looking at the data

The data is included in `scikit-learn` and so can be loaded easily.

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
digits.images.shape

As usual, we review the training data before doing anything else. In this case, the data takes the form of rasterised images, so it makes sense to display them as such, overlaying each image with the label it was assigned by a human.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axes = plt.subplots(10, 10, figsize=(8, 8))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary', interpolation='nearest')
    ax.text(0.05, 0.05, str(digits.target[i]),
            transform=ax.transAxes, color='green')
    ax.set_xticks([])
    ax.set_yticks([])

Here the data per digit is simply each pixel value within an 8x8 grid. The example grid below represents a zero.

In [ ]:
# The images themselves
print(digits.images.shape)
print(digits.images[0])

While it is better to display each instance as an 8x8 grid, each instance needs to be flattened into a single row with 64 elements (columns), as below.

In [ ]:
# The flattened data that is used to train the model.
print(digits.data.shape)
print(digits.data[0])

There are some nice facilities to count the number of different digits.

In [ ]:
# The target label
from collections import Counter # https://stackoverflow.com/a/2392948
c = Counter(digits.target)

print(c.items())

Summarising, the data has 1797 samples in 64 dimensions and 10 ($0,\ldots,9$) levels. The number of instances per level varies from 174 to 183.

## Classifying the digits using logistic regression

Logistic regression is an extension of regression where a change of variable is used to map the continuous (numerical-valued) prediction into categorical values for classification purposes.

In [ ]:
from sklearn.model_selection import train_test_split
seed=2
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=seed, stratify=digits.target)
print(Xtrain.shape, Xtest.shape)

We use logistic regression to classify the scanned written digits

In [ ]:
from sklearn.linear_model import LogisticRegression

# Get and configure a LogisticRegression object
clf = LogisticRegression(max_iter=7600)

# Fit the training data
clf.fit(Xtrain, ytrain)

# Using the beta parameters that have been learned and are in clf, predict (recognise) the test data
ypred = clf.predict(Xtest)

We check the classification accuracy score and confusion matrix.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(ytest, ypred))
confusionMat = confusion_matrix(ytest, ypred)
print(confusionMat)

As can be seen, the confusion matrix has several off-diagonal nonzero terms. Because there are 10 labels, the confusion matrix is slightly harder to visualise than the Iris data, which had just 3 labels. We can get a better sense of its layout by plotting it as an image. Because all the values are nonnegative, but there is a large difference in size from the smallest (0) to the largest (45) with most values being at each end of the range, the square root of the values maps better into the Blue colour space used in the plot below:

Make sure the directory exists beforehand to store the generated plots

In [ ]:
import os
picDir = "output/LR"
if not os.path.exists(picDir):
    os.makedirs(picDir)

In [ ]:
import numpy as np
plt.imshow(np.sqrt(confusionMat),cmap='Blues', interpolation='nearest')
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted');
plt.savefig(picDir+"/logreg_digits_confusionMatrix.pdf")

We might also take a look at some of the outputs along with their predicted labels. Matching labels are <font color='green'>green</font> (as before) and unmatched labels are <font color='red'>red</font>:

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(8, 8))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, ax in enumerate(axes.flat):
    ax.imshow(Xtest[i].reshape(8, 8), cmap='binary')
    ax.text(0.05, 0.05, str(ypred[i]),
            transform=ax.transAxes,
            color='green' if (ytest[i] == ypred[i]) else 'red')
    ax.set_xticks([])
    ax.set_yticks([])
fig.savefig(picDir+"/digitsAccuracyCheck.pdf")

Where they do not match, it is arguable what the original writing was meant to represent!

## Comments

Logistic Regression using all $8\times 8 = 64$ pixel values per image might not be very efficient. At the very least there is probably correlation between the 64 features. This will show up as the solver needing many iterations (taking a lot of time) or even failing to converge.

With such image data, how would you chose which features (pixels in this case) to keep in the model?

An alternative approach is to look for an uncorrelated set of features that capture most of the variance explained by the original 64 features. This makes the solver's task easier, but might weaken the predictive power of the model. In practice the parameter space associated with the uncorrelated set of features generally has much lower dimension than the original set of features from which they were derived. This procedures is known as _dimensionality reduction_ and is the subject of the other notebook in today's practical.

## Exercise

1. Use Principal Components Analysis (PCA) to derive a lower dimensional set of features, and apply Logistic Regression to the resulting reduced dimension dataframe/model.
2. Try using [GridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find the optimal setting of PCA's `n_components` parameter. You could search over the range 10 to 40, say.